## Imports / Globals

In [1]:
import pickle
import pandas as pd
from os import listdir
from os.path import isfile, join
from tqdm import tqdm
import numpy as np
from sklearn.model_selection import train_test_split

np.random.seed(42)

In [2]:
# path = '/Users/thomas/Downloads/nturgb+d_skeletons'
path = 'D:\\Datasets\\Motion Privacy\\NTU RGB+D 120\\Skeleton Data'

## Data organization

### Input format
needs to be updated

[J0 X, J0 Y, J0 Z, J1 X, J1 Y, J1 Z, ..., J25 Z]

### Output format
0 = Female, 1 = Male

In [3]:
# Load the Genders
Genders = pd.read_csv('data/Genders.csv')

# Convert M to 1 and F to 0
Genders = Genders.replace('M', 1).replace('F', 0)

# Convert dataframe to oject where P is the key, and Gender is the value
Genders = Genders.set_index('P').T.to_dict('list')

In [4]:
def load_files():
    # Read the files
    files = [f for f in listdir(path) if isfile(join(path, f))]

    # Get stats for each file based on name
    files_ = []
    for file in files:
        data = {'file': file,
                's': file[0:4],
                'c': file[4:8],
                'p': file[8:12],
                'r': file[12:16],
                'a': file[16:20]
                }
        files_.append(data)

    return files_
files_ = None

In [5]:
# Attempt to load X and Y from pickle before generating them
X = {}
try:
    print('Attempting to load X from pickle')
    with open('data/X.pkl', 'rb') as f:
        X = pickle.load(f)
    print('X loaded from pickle')
except:
    print('Could not load X and Y, generating them now')

    # Load the files
    if files_ is None:
        files_ = load_files()

    # Generate X and Y
    for file_ in tqdm(files_, desc='Files Parsed', position=0):
        try:
            file = join(path, file_['file'])
            data = open(file, 'r')
            lines = data.readlines()
            frames_count = int(lines.pop(0).replace('\n', ''))
            file_['frames'] = frames_count
        except UnicodeDecodeError:  # .DS_Store file
            print('UnicodeDecodeError: ', file)
            continue

        # Get P and add to X if not already there
        p = file_['p']
        if p not in X:
            X[p] = []

        # Hold all frames for the file (action)
        frames = []

        # To validate the video is good
        good = True

        for f in tqdm(range(frames_count), desc='Frames Parsed', position=1, leave=False):
            try:
                # Get actor count
                actors = int(lines.pop(0).replace('\n', ''))

                # Hold frame info
                frame = []

                # Iterate through actors
                for a in range(actors):
                    # Get actor info
                    t = lines.pop(0)

                    # Get joint count
                    joint_count = int(lines.pop(0).replace('\n', ''))

                    # Get joint info
                    d = []
                    for j in range(joint_count):
                        joint = lines.pop(0).replace('\n', '').split(' ')
                        d.extend(joint[0:3])

                    # Convert to float
                    d = [float(i) for i in d]

                    # Skip if not 25 joints
                    if len(d) != 75:
                        good = False
                        break

                    # Append to frame
                    frame.extend(d)

                # Convert to numpy array
                frame = np.array(frame, dtype=np.float32)

                # Pad frame to 150 (for 2 actors)
                if len(frame) < 150:
                    frame = np.pad(frame, (0, 150-len(frame)), 'constant')

                # Append to X and Y
                frames.append(frame)
            except:
                break

        if not good:
            continue

        if type(frames) != list:
            print('Not a list: ', type(frames), frames)
            continue

        # Convert to numpy array
        frames = np.array(frames, dtype=np.ndarray)

        # Validate frames size
        if len(frames.shape) != 2 or frames.shape[1] < 150:
            continue

        # Pad X size to 300 frames (300 is max frames in dataset)
        # Each frame is 25 joints * 3 coordinates * 2 potential actors = 150
        # For the real time attack model, we can make a new prediction every 300 frames (5 second @ 60fps) to align with this
        if frames.shape[0] < 300:
            frames = np.pad(
                frames, ((0, 300-frames.shape[0]), (0, 0)), 'constant')

        # Validate frames size
        if frames.shape != (300, 150):
            continue

        # Ensure the frame isnt all zeros
        if np.sum(frames) == 0:
            continue

        # Add frames to X
        X[p].append(frames)

    # Convert to numpy arrays
    for p in X:
        X[p] = np.array(X[p]).astype(np.float32)

    print('X Generated, saving to pickle...')

    # Save the data
    with open('data/X.pkl', 'wb') as f:
        pickle.dump(X, f)

    print('X Saved to pickle')

# Print Lengths
# for p in X:
#     print(p, len(X[p]))

Attempting to load X from pickle
X loaded from pickle


In [6]:
# Default Male/Female split
MALES = 30
FEMALES = 15

# To export test actors
test_actors = []

# Split into train and test
def split_data(males, females):
    global test_actors
    # Choose males count random males
    m = []
    f = []
    for p in Genders:
        if Genders[p] == [1]:
            m.append(p)
        else:
            f.append(p)
    train_males = np.random.choice(list(m), males, replace=False)
    train_females = np.random.choice(list(f), females, replace=False)

    # Combine to get male/female split
    X_male = np.concatenate([X[x] for x in train_males])
    X_female = np.concatenate([X[x] for x in train_females])
    Y_male = np.ones(len(X_male))
    Y_female = np.zeros(len(X_female))

    # Combine to get train data
    train_x = np.concatenate([X_male, X_female])
    train_y = np.concatenate([Y_male, Y_female])

    # Get the actors not in the train set
    test_males = [x for x in m if x not in train_males]
    test_females = [x for x in f if x not in train_females]
    test_actors = test_males+test_females

    # Combine to get male/female split
    X_male = np.concatenate([X[x] for x in test_males])
    X_female = np.concatenate([X[x] for x in test_females])
    Y_male = np.ones(len(X_male), dtype=np.int8)
    Y_female = np.zeros(len(X_female), dtype=np.int8)

    # Combine to get test data
    test_x = np.concatenate([X_male, X_female])
    test_y = np.concatenate([Y_male, Y_female])

    # Print shapes
    print('Train X Shape: ', train_x.shape)
    print('Train Y Shape: ', train_y.shape)
    print('Test X Shape: ', test_x.shape)
    print('Test Y Shape: ', test_y.shape)

    return train_x, train_y, test_x, test_y

train_x, train_y, test_x, test_y = split_data(MALES, FEMALES)

Train X Shape:  (55641, 300, 150)
Train Y Shape:  (55641,)
Test X Shape:  (57336, 300, 150)
Test Y Shape:  (57336,)


In [7]:
# Split test into validation and test
test_x, val_x, test_y, val_y = train_test_split(test_x, test_y, test_size=0.5, random_state=42)

In [8]:
# Update all Y's to be onehot
def to_categorical(y):
    return np.array([np.array([1, 0]) if i == 0 else np.array([0, 1]) for i in y])
train_y = to_categorical(train_y)
test_y = to_categorical(test_y)
val_y = to_categorical(val_y)

In [9]:
# Save Test Actors
test_X = {}
for p in test_actors:
    test_X[p] = X[p]

with open('data/test_X.pkl', 'wb') as f:
    pickle.dump(test_X, f)

## SGN

All code in this section is adapted from Microsoft's SGN. [Github](https://github.com/microsoft/SGN)

In [10]:
import time
import shutil
import os
import os.path as osp
import csv
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import MultiStepLR
from model import SGN
from data import NTUDataLoaders, AverageMeter
from util import make_dir, get_num_classes

In [11]:
# Hyperparameters/Tuning Parameters
network='SGN'
dataset='NTU'
start_epoch=0
case=0
batch_size=64
max_epochs=120
monitor='val_acc'
lr=0.001
weight_decay=0.0001
lr_factor=0.1
workers=16
print_freq = 20
do_train=1
seg=20

In [16]:
def train(train_loader, model, criterion, optimizer, epoch):
    losses = AverageMeter()
    acces = AverageMeter()
    model.train()

    for i, (inputs, target) in enumerate(train_loader):

        output = model(inputs.cuda())
        target = target.cuda()
        loss = criterion(output, target)

        # measure accuracy and record loss
        acc = accuracy(output.data, target)
        losses.update(loss.item(), inputs.size(0))
        acces.update(acc[0], inputs.size(0))

        # backward
        optimizer.zero_grad()  # clear gradients out before each mini-batch
        loss.backward()
        optimizer.step()

        if (i + 1) % print_freq == 0:
            print('Epoch-{:<3d} {:3d} batches\t'
                  'loss {loss.val:.4f} ({loss.avg:.4f})\t'
                  'accu {acc.val:.3f} ({acc.avg:.3f})'.format(
                      epoch + 1, i + 1, loss=losses, acc=acces))

    return losses.avg, acces.avg


def validate(val_loader, model, criterion):
    losses = AverageMeter()
    acces = AverageMeter()
    model.eval()

    for i, (inputs, target) in enumerate(val_loader):
        with torch.no_grad():
            output = model(inputs.cuda())
        target = target.cuda()
        with torch.no_grad():
            loss = criterion(output, target)

        # measure accuracy and record loss
        acc = accuracy(output.data, target)
        losses.update(loss.item(), inputs.size(0))
        acces.update(acc[0], inputs.size(0))

    return losses.avg, acces.avg


def test(test_loader, model, checkpoint, lable_path, pred_path):
    acces = AverageMeter()
    # load learnt model that obtained best performance on validation set
    model.load_state_dict(torch.load(checkpoint)['state_dict'])
    model.eval()

    label_output = list()
    pred_output = list()

    t_start = time.time()
    for i, t in enumerate(test_loader):
        inputs = t[0]
        target = t[1]
        with torch.no_grad():
            output = model(inputs.cuda())
            output = output.view(
                (-1, inputs.size(0)//target.size(0), output.size(1)))
            output = output.mean(1)

        label_output.append(target.cpu().numpy())
        pred_output.append(output.cpu().numpy())

        acc = accuracy(output.data, target.cuda())
        acces.update(acc[0], inputs.size(0))

    label_output = np.concatenate(label_output, axis=0)
    np.savetxt(lable_path, label_output, fmt='%d')
    pred_output = np.concatenate(pred_output, axis=0)
    np.savetxt(pred_path, pred_output, fmt='%f')

    print('Test: accuracy {:.3f}, time: {:.2f}s'
          .format(acces.avg, time.time() - t_start))


def accuracy(output, target):
    batch_size = target.size(0)
    _, pred = output.topk(1, 1, True, True)
    pred = pred.t()
    target = torch.argmax(target, dim=1)  # Add this line to convert one-hot targets to class indices
    correct = pred.eq(target.view(1, -1).expand_as(pred))
    correct = correct.view(-1).float().sum(0, keepdim=True)
    return correct.mul_(100.0 / batch_size)



def save_checkpoint(state, filename='checkpoint.pth.tar', is_best=False):
    torch.save(state, filename)
    if is_best:
        shutil.copyfile(filename, 'model_best.pth.tar')


def get_n_params(model):
    pp = 0
    for p in list(model.parameters()):
        nn = 1
        for s in list(p.size()):
            nn = nn*s
        pp += nn
    return pp


class LabelSmoothingLoss(nn.Module):
    def __init__(self, classes, smoothing=0.0, dim=-1):
        super(LabelSmoothingLoss, self).__init__()
        self.confidence = 1.0 - smoothing
        self.smoothing = smoothing
        self.cls = classes
        self.dim = dim

    def forward(self, pred, target):
        pred = pred.log_softmax(dim=self.dim)
        with torch.no_grad():
            target = torch.argmax(target, dim=1)  # Add this line to convert one-hot targets to class indices
            true_dist = torch.zeros_like(pred)
            true_dist.fill_(self.smoothing / (self.cls - 1))
            true_dist.scatter_(1, target.data.unsqueeze(1), self.confidence)
        return torch.mean(torch.sum(-true_dist * pred, dim=self.dim))



In [17]:
def main():
    num_classes = get_num_classes(dataset)
    model = SGN(num_classes, dataset, seg, batch_size, do_train)

    total = get_n_params(model)
    # print(model)
    print('The number of parameters: ', total)
    print('The modes is:', network)

    if torch.cuda.is_available():
        print('It is using GPU!')
        model = model.cuda()

    criterion = LabelSmoothingLoss(num_classes, smoothing=0.1).cuda()
    optimizer = optim.Adam(model.parameters(), lr=lr,
                           weight_decay=weight_decay)

    if monitor == 'val_acc':
        mode = 'max'
        monitor_op = np.greater
        best = -np.Inf
        str_op = 'improve'
    elif monitor == 'val_loss':
        mode = 'min'
        monitor_op = np.less
        best = np.Inf
        str_op = 'reduce'

    scheduler = MultiStepLR(optimizer, milestones=[60, 90, 110], gamma=0.1)
    # Data loading
    ntu_loaders = NTUDataLoaders(dataset, case, seg=seg, train_X=train_x, train_Y=train_y, test_X=test_x, test_Y=test_y, val_X=val_x, val_Y=val_y, aug=0)
    train_loader = ntu_loaders.get_train_loader(batch_size, workers)
    val_loader = ntu_loaders.get_val_loader(batch_size, workers)
    train_size = ntu_loaders.get_train_size()
    val_size = ntu_loaders.get_val_size()

    test_loader = ntu_loaders.get_test_loader(32, workers)

    print('Train on %d samples, validate on %d samples' %
          (train_size, val_size))

    best_epoch = 0
    output_dir = make_dir(dataset)

    save_path = os.path.join(output_dir, network)
    if not os.path.exists(save_path):
        os.makedirs(save_path)

    checkpoint = osp.join(save_path, '%s_best.pth' % case)
    earlystop_cnt = 0
    csv_file = osp.join(save_path, '%s_log.csv' % case)
    log_res = list()

    lable_path = osp.join(save_path, '%s_lable.txt' % case)
    pred_path = osp.join(save_path, '%s_pred.txt' % case)

    # Training
    if do_train == 1:
        for epoch in range(start_epoch, max_epochs):

            print(epoch, optimizer.param_groups[0]['lr'])

            t_start = time.time()
            train_loss, train_acc = train(
                train_loader, model, criterion, optimizer, epoch)
            val_loss, val_acc = validate(val_loader, model, criterion)
            log_res += [[train_loss, train_acc.cpu().numpy(),
                         val_loss, val_acc.cpu().numpy()]]

            print('Epoch-{:<3d} {:.1f}s\t'
                  'Train: loss {:.4f}\taccu {:.4f}\tValid: loss {:.4f}\taccu {:.4f}'
                  .format(epoch + 1, time.time() - t_start, train_loss, train_acc, val_loss, val_acc))

            current = val_loss if mode == 'min' else val_acc

            # store tensor in cpu
            current = current.cpu()

            if monitor_op(current, best):
                print('Epoch %d: %s %sd from %.4f to %.4f, '
                      'saving model to %s'
                      % (epoch + 1, monitor, str_op, best, current, checkpoint))
                best = current
                best_epoch = epoch + 1
                save_checkpoint({
                    'epoch': epoch + 1,
                    'state_dict': model.state_dict(),
                    'best': best,
                    'monitor': monitor,
                    'optimizer': optimizer.state_dict(),
                }, checkpoint)
                earlystop_cnt = 0
            else:
                print('Epoch %d: %s did not %s' % (epoch + 1, monitor, str_op))
                earlystop_cnt += 1

            scheduler.step()

        print('Best %s: %.4f from epoch-%d' % (monitor, best, best_epoch))
        with open(csv_file, 'w') as fw:
            cw = csv.writer(fw)
            cw.writerow(['loss', 'acc', 'val_loss', 'val_acc'])
            cw.writerows(log_res)
        print('Save train and validation log into into %s' % csv_file)

    # Test
    model = SGN(num_classes, dataset, seg, batch_size, 0)
    model = model.cuda()
    test(test_loader, model, checkpoint, lable_path, pred_path)

main()

The number of parameters:  661294
The modes is: SGN
It is using GPU!
Train on 55641 samples, validate on 28668 samples
0 0.001
Epoch-1    20 batches	loss 0.6455 (0.7665)	accu 73.438 (64.531)
Epoch-1    40 batches	loss 0.7897 (0.7195)	accu 59.375 (65.508)
Epoch-1    60 batches	loss 0.6511 (0.6917)	accu 70.312 (67.266)
Epoch-1    80 batches	loss 0.6671 (0.6738)	accu 71.875 (68.633)
Epoch-1   100 batches	loss 0.5672 (0.6635)	accu 76.562 (69.156)
Epoch-1   120 batches	loss 0.5462 (0.6501)	accu 82.812 (70.312)
Epoch-1   140 batches	loss 0.5938 (0.6421)	accu 79.688 (70.960)
Epoch-1   160 batches	loss 0.5389 (0.6340)	accu 82.812 (71.592)
Epoch-1   180 batches	loss 0.5840 (0.6258)	accu 75.000 (72.405)
Epoch-1   200 batches	loss 0.5645 (0.6164)	accu 78.125 (73.312)
Epoch-1   220 batches	loss 0.5732 (0.6106)	accu 82.812 (73.828)
Epoch-1   240 batches	loss 0.5919 (0.6088)	accu 78.125 (73.809)
Epoch-1   260 batches	loss 0.4942 (0.6037)	accu 84.375 (74.417)
Epoch-1   280 batches	loss 0.5890 (0.6011

KeyboardInterrupt: 